# Modelo de programación lineal

## Entrada de datos: Instancia 2x100x1.5x1

In [1]:
import pandas as pd
import gurobipy as grb

In [2]:
datafile="2x100x1.5x1.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

100

## Modelo

In [3]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])
    
ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Academic license - for non-commercial use only - expires 2021-06-11
Using license file C:\Users\elPatan03\gurobi.lic
Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 707500 rows, 60000 columns and 4543400 nonzeros
Model fingerprint: 0xbcbfa4f3
Variable types: 0 continuous, 60000 integer (60000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 9.0000000
Presolve removed 657621 rows and 54090 columns (presolve time = 5s) ...
Presolve removed 699972 rows and 56254 columns
Presolve time: 9.75s
Presolved: 7528 rows, 3746 columns, 142067 nonzeros
Variable types: 0 continuous, 3746 integer (3684 binar

## Entrada de datos: Instancia 2x100x1.5x2

In [4]:
datafile="2x100x1.5x2.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

100

In [5]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])
    
ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 707500 rows, 60000 columns and 4543400 nonzeros
Model fingerprint: 0x8d075d89
Variable types: 0 continuous, 60000 integer (60000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 10.0000000
Presolve removed 652741 rows and 54462 columns (presolve time = 5s) ...
Presolve removed 698904 rows and 56668 columns
Presolve time: 9.73s
Presolved: 8596 rows, 3332 columns, 190140 nonzeros
Found heuristic solution: objective 83.0000000
Variable types: 0 continuous, 3332 integer (3312 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    

## Entrada de datos: Instancia 2x100x1.5x3

In [6]:
datafile="2x100x1.5x3.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

100

In [7]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 707500 rows, 60000 columns and 4543400 nonzeros
Model fingerprint: 0x05cc823f
Variable types: 0 continuous, 60000 integer (60000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 7.0000000
Presolve removed 662081 rows and 54370 columns (presolve time = 5s) ...
Presolve removed 698793 rows and 56508 columns
Presolve time: 9.79s
Presolved: 8707 rows, 3492 columns, 175879 nonzeros
Found heuristic solution: objective 73.0000000
Variable types: 0 continuous, 3492 integer (3456 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    D

## Entrada de datos: Instancia 2x100x1.5x4

In [8]:
datafile="2x100x1.5x4.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

100

In [9]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 707500 rows, 60000 columns and 4543400 nonzeros
Model fingerprint: 0x062c3000
Variable types: 0 continuous, 60000 integer (60000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 8.0000000
Presolve removed 658236 rows and 54470 columns (presolve time = 5s) ...
Presolve removed 698488 rows and 56578 columns
Presolve time: 9.49s
Presolved: 9012 rows, 3422 columns, 184811 nonzeros
Variable types: 0 continuous, 3422 integer (3408 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.0600000e+02   

## Entrada de datos: Instancia 2x100x1.5x5

In [10]:
datafile="2x100x1.5x5.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

100

In [11]:

p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])
    


ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 707500 rows, 60000 columns and 4543400 nonzeros
Model fingerprint: 0x99e2feb4
Variable types: 0 continuous, 60000 integer (60000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 8.0000000
Presolve removed 666305 rows and 54714 columns (presolve time = 5s) ...
Presolve removed 699835 rows and 56938 columns
Presolve time: 8.38s
Presolved: 7665 rows, 3062 columns, 186986 nonzeros
Found heuristic solution: objective 71.0000000
Variable types: 0 continuous, 3062 integer (3030 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    D

## Entrada de datos: Instancia 2x100x2x1

In [12]:
datafile="2x100x2x1.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

100

In [13]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]


#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 707500 rows, 60000 columns and 4543400 nonzeros
Model fingerprint: 0x1a2f5ebc
Variable types: 0 continuous, 60000 integer (60000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 8.0000000
Presolve removed 680823 rows and 56104 columns (presolve time = 5s) ...
Presolve removed 705291 rows and 57651 columns
Presolve time: 6.65s
Presolved: 2209 rows, 2349 columns, 53977 nonzeros
Found heuristic solution: objective 60.0000000
Variable types: 0 continuous, 2349 integer (2335 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Du

## Entrada de datos: Instancia 2x100x2x2

In [14]:
datafile="2x100x2x2.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

100

In [15]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])
    
ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 707500 rows, 60000 columns and 4543400 nonzeros
Model fingerprint: 0xc84eeeca
Variable types: 0 continuous, 60000 integer (60000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 8.0000000
Presolve removed 669407 rows and 55416 columns (presolve time = 5s) ...
Presolve removed 700552 rows and 57110 columns
Presolve time: 8.69s
Presolved: 6948 rows, 2890 columns, 141640 nonzeros
Variable types: 0 continuous, 2890 integer (2832 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.6300000e+02   

 146316  8359   77.00000  138   47   76.00000   77.00000  1.32%  68.4  425s
 147360  8902   77.00000  188   28   76.00000   77.00000  1.32%  68.3  430s
 151853 10135   77.00000  154   18   76.00000   77.00000  1.32%  68.1  436s
 153069 10353   77.00000  157   36   76.00000   77.00000  1.32%  68.2  440s
 155302 10656   77.00000  146   29   76.00000   77.00000  1.32%  68.2  445s
 158090 10979   77.00000  143   18   76.00000   77.00000  1.32%  68.2  450s
 160669 11326   77.00000  136   24   76.00000   77.00000  1.32%  68.5  456s
 161786 11479   77.00000  136   21   76.00000   77.00000  1.32%  69.0  461s
 163922 12101   77.00000  154   27   76.00000   77.00000  1.32%  69.4  466s
 165253 12334 infeasible  144        76.00000   77.00000  1.32%  69.9  472s
 166177 12450   77.00000  163   45   76.00000   77.00000  1.32%  70.2  476s
 167895 12843   77.00000  177   22   76.00000   77.00000  1.32%  70.8  482s
 168787 12985   77.00000  142   54   76.00000   77.00000  1.32%  71.1  485s
 171074 1341

 331181 27076   77.00000  154   37   76.00000   77.00000  1.32%  84.6  971s
 332840 27152   77.00000  139   37   76.00000   77.00000  1.32%  84.6  975s
 335284 27341 infeasible  167        76.00000   77.00000  1.32%  84.5  981s
 337277 27373   77.00000  165   32   76.00000   77.00000  1.32%  84.4  985s
 339019 27458 infeasible  143        76.00000   77.00000  1.32%  84.4  990s
 341142 27540   77.00000  139   35   76.00000   77.00000  1.32%  84.4  995s
 342991 27525 infeasible  137        76.00000   77.00000  1.32%  84.3 1000s
 346149 27564   77.00000  149   37   76.00000   77.00000  1.32%  84.3 1006s
 348158 27649 infeasible  152        76.00000   77.00000  1.32%  84.3 1011s
 349820 27683   77.00000  145   30   76.00000   77.00000  1.32%  84.3 1015s
 351730 27769   77.00000  164   40   76.00000   77.00000  1.32%  84.3 1020s
 354006 27872 infeasible  129        76.00000   77.00000  1.32%  84.2 1025s
 356014 28003   77.00000  121   40   76.00000   77.00000  1.32%  84.1 1030s
 357522 2803

 507793 33555 infeasible  159        76.00000   77.00000  1.32%  90.0 1511s
 510094 33544 infeasible  181        76.00000   77.00000  1.32%  90.1 1517s
 512201 33635 infeasible  151        76.00000   77.00000  1.32%  90.1 1523s
 512942 33631   77.00000  170   19   76.00000   77.00000  1.32%  90.1 1526s
 514992 33657 infeasible  156        76.00000   77.00000  1.32%  90.1 1532s
 517001 33637   77.00000  178   19   76.00000   77.00000  1.32%  90.1 1535s
 519457 33779 infeasible  151        76.00000   77.00000  1.32%  90.1 1542s
 521764 33821   77.00000  156   32   76.00000   77.00000  1.32%  90.1 1548s
 522529 33927   77.00000  154   18   76.00000   77.00000  1.32%  90.1 1550s
 524575 33915   77.00000  192   17   76.00000   77.00000  1.32%  90.1 1555s
 527427 33863   77.00000  158   39   76.00000   77.00000  1.32%  90.2 1562s
 528021 33830   77.00000  151   51   76.00000   77.00000  1.32%  90.2 1565s
 529507 33811     cutoff  152        76.00000   77.00000  1.32%  90.3 1572s
 531050 3373

 701226 34595   77.00000  151   55   76.00000   77.00000  1.32%  92.6 2061s
 702772 34600   77.00000  135   46   76.00000   77.00000  1.32%  92.6 2066s
 704887 34634   77.00000  151   57   76.00000   77.00000  1.32%  92.7 2073s
 705901 34574   77.00000  151   25   76.00000   77.00000  1.32%  92.7 2076s
 707623 34559   77.00000  186   39   76.00000   77.00000  1.32%  92.8 2082s
 709429 34553     cutoff  143        76.00000   77.00000  1.32%  92.9 2087s
 710195 34555 infeasible  161        76.00000   77.00000  1.32%  92.9 2090s
 711556 34511   77.00000  155   68   76.00000   77.00000  1.32%  93.0 2098s
 713201 34588 infeasible  159        76.00000   77.00000  1.32%  93.0 2101s
 715165 34607 infeasible  165        76.00000   77.00000  1.32%  93.1 2107s
 715689 34568   77.00000  139   45   76.00000   77.00000  1.32%  93.1 2111s
 717933 34472 infeasible  144        76.00000   77.00000  1.32%  93.2 2117s
 718553 34421     cutoff  154        76.00000   77.00000  1.32%  93.2 2120s
 720518 3440

 848939 37396   77.00000  128   50   76.00000   77.00000  1.32%  98.4 2602s
 850241 37463   77.00000  152   38   76.00000   77.00000  1.32%  98.4 2606s
 851616 37651     cutoff  144        76.00000   77.00000  1.32%  98.5 2613s
 852913 37771 infeasible  123        76.00000   77.00000  1.32%  98.6 2617s
 853959 37813 infeasible  153        76.00000   77.00000  1.32%  98.6 2621s
 855927 37913   77.00000  133   33   76.00000   77.00000  1.32%  98.7 2628s
 856772 37986 infeasible  153        76.00000   77.00000  1.32%  98.8 2632s
 857674 38071   77.00000  166   63   76.00000   77.00000  1.32%  98.8 2636s
 859103 38201     cutoff  142        76.00000   77.00000  1.32%  98.9 2644s
 860629 38316     cutoff  130        76.00000   77.00000  1.32%  99.0 2647s
 861514 38408     cutoff  148        76.00000   77.00000  1.32%  99.0 2651s
 863370 38496   77.00000  154   18   76.00000   77.00000  1.32%  99.2 2658s
 863893 38553   77.00000  161   48   76.00000   77.00000  1.32%  99.2 2661s
 864981 3862

 988492 39612     cutoff  205        76.00000   77.00000  1.32%   107 3145s
 991538 39599   77.00000  143   34   76.00000   77.00000  1.32%   107 3150s
 993674 39611   77.00000  149   60   76.00000   77.00000  1.32%   106 3155s
 995650 39585   77.00000  196   24   76.00000   77.00000  1.32%   106 3160s
 997798 39575   77.00000  202   28   76.00000   77.00000  1.32%   106 3165s
 1000272 39584 infeasible  214        76.00000   77.00000  1.32%   106 3170s
 1002439 39522 infeasible  184        76.00000   77.00000  1.32%   106 3176s
 1004941 39473     cutoff  187        76.00000   77.00000  1.32%   106 3181s
 1006141 39465 infeasible  184        76.00000   77.00000  1.32%   106 3185s
 1008343 39575   77.00000  190   33   76.00000   77.00000  1.32%   106 3191s
 1009678 39596 infeasible  189        76.00000   77.00000  1.32%   106 3195s
 1010974 39622   77.00000  162   46   76.00000   77.00000  1.32%   106 3200s
 1012680 39527   77.00000  184   31   76.00000   77.00000  1.32%   106 3206s
 101

## Entrada de datos: Instancia 2x100x2x3

In [16]:
datafile="2x100x2x3.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

100

In [17]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])
    
ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 707500 rows, 60000 columns and 4543400 nonzeros
Model fingerprint: 0x6ffaa178
Variable types: 0 continuous, 60000 integer (60000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 7.0000000
Presolve removed 677159 rows and 55778 columns (presolve time = 5s) ...
Presolve removed 704758 rows and 57434 columns
Presolve time: 7.58s
Presolved: 2742 rows, 2566 columns, 58904 nonzeros
Found heuristic solution: objective 63.0000000
Variable types: 0 continuous, 2566 integer (2524 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Du

## Entrada de datos: Instancia 2x100x2x4

In [18]:
datafile="2x100x2x4.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

100

In [19]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])
    
ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 707500 rows, 60000 columns and 4543400 nonzeros
Model fingerprint: 0x2c7c7b7b
Variable types: 0 continuous, 60000 integer (60000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 8.0000000
Presolve removed 674962 rows and 55808 columns (presolve time = 5s) ...
Presolve removed 704357 rows and 57344 columns
Presolve time: 7.32s
Presolved: 3143 rows, 2656 columns, 76741 nonzeros
Variable types: 0 continuous, 2656 integer (2644 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.1600000e+02   1

  2081  1610   80.42317   79  242   79.00000   80.42317  1.80%   426  315s
  2100  1624   80.42229  110  117   79.00000   80.42229  1.80%   435  320s
  2109  1630   80.25378   27  164   79.00000   80.42229  1.80%   433  325s
  2121  1638   80.42148   50  280   79.00000   80.42148  1.80%   431  330s
  2136  1650   80.42123   24  241   79.00000   80.42123  1.80%   441  335s
  2150  1659   80.42053   35  287   79.00000   80.42053  1.80%   438  340s
  2164  1670   80.42053   13  207   79.00000   80.42053  1.80%   446  345s
  2178  1679   80.41957    9  231   79.00000   80.41957  1.80%   443  350s
  2192  1689   80.41539    8  290   79.00000   80.41539  1.79%   440  355s
  2206  1698   80.41139    5  302   79.00000   80.41139  1.79%   437  360s
  2224  1711   80.41017   39  225   79.00000   80.41017  1.79%   453  365s
  2233  1717   80.41013   23  264   79.00000   80.41013  1.78%   451  370s
  2250  1730   80.40972   35  229   79.00000   80.40972  1.78%   458  375s
  2259  1736   80.40972  

 26493 18869   80.24178  461  249   79.00000   80.36846  1.73%   211  942s
 26494 18872   80.36846  358  250   79.00000   80.36846  1.73%   215  945s
 26496 18876   80.36846  359  223   79.00000   80.36846  1.73%   215  950s
 26506 18884   80.36846  361  251   79.00000   80.36846  1.73%   215  957s
 26518 18892   80.36846  362  260   79.00000   80.36846  1.73%   216  961s
 26524 18896   80.33991  362  238   79.00000   80.36846  1.73%   216  965s
 26555 18919   80.32846  365  251   79.00000   80.36846  1.73%   216  971s
 26590 18939   80.09407  368  173   79.00000   80.36846  1.73%   217  979s
 26596 18957   80.36097  369  279   79.00000   80.36846  1.73%   217  980s
 26667 19007   80.35832  375  265   79.00000   80.36846  1.73%   219  985s
 26804 19102     cutoff  383        79.00000   80.36846  1.73%   220  990s
 26976 19200   80.34428  393  244   79.00000   80.36846  1.73%   222  995s
 27031 19281   80.34302  396  240   79.00000   80.36846  1.73%   222 1000s
 27385 19500   80.32511  

 97849 53202   80.24783  396  220   79.00000   80.31763  1.67%   319 2073s
 98520 53663   80.17147  438  198   79.00000   80.31763  1.67%   321 2087s
 99289 54191   80.08081  524  117   79.00000   80.31717  1.67%   321 2102s
 100126 54914   80.06208  383  225   79.00000   80.31668  1.67%   322 2117s
 101104 55742   80.20225  405  258   79.00000   80.31628  1.67%   323 2132s
 102142 56377   80.28796  378  344   79.00000   80.31597  1.67%   323 2147s
 103021 56848   80.18035  431  265   79.00000   80.31584  1.67%   323 2162s
 103651 57236   80.20350  412  202   79.00000   80.31569  1.67%   324 2177s
 104185 57748   80.22143  416  177   79.00000   80.31554  1.67%   325 2192s
 104909 58525   80.16807  463  149   79.00000   80.31553  1.67%   327 2209s
 106330 59215   80.16639  405  215   79.00000   80.31533  1.66%   327 2222s
 107174 59597   80.10678  467  130   79.00000   80.31509  1.66%   327 2235s
 107775 60285   80.02028  389  194   79.00000   80.31493  1.66%   328 2249s
 108645 60595  

## Entrada de datos: Instancia 2x100x2x5

In [20]:
datafile="2x100x2x5.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

100

In [21]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])
    
ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 707500 rows, 60000 columns and 4543400 nonzeros
Model fingerprint: 0xc894d3d2
Variable types: 0 continuous, 60000 integer (60000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 8.0000000
Presolve removed 673063 rows and 55540 columns (presolve time = 5s) ...
Presolve removed 700941 rows and 57154 columns
Presolve time: 8.95s
Presolved: 6559 rows, 2846 columns, 138788 nonzeros
Variable types: 0 continuous, 2846 integer (2812 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.2100000e+02   

## Entrada de datos: Instancia 2x100x2.5x1

In [22]:
datafile="2x100x2.5x1.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

100

In [23]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])
    
ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 707500 rows, 60000 columns and 4543400 nonzeros
Model fingerprint: 0xc1b2fd94
Variable types: 0 continuous, 60000 integer (60000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 6.0000000
Presolve removed 681619 rows and 56558 columns (presolve time = 5s) ...
Presolve removed 705090 rows and 57798 columns
Presolve time: 7.67s
Presolved: 2410 rows, 2202 columns, 59816 nonzeros
Variable types: 0 continuous, 2202 integer (2180 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.9500000e+02   8

## Entrada de datos: Instancia 2x100x2.5x2

In [24]:
datafile="2x100x2.5x2.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

100

In [25]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])
    
ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 707500 rows, 60000 columns and 4543400 nonzeros
Model fingerprint: 0x7f41adb2
Variable types: 0 continuous, 60000 integer (60000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 7.0000000
Presolve removed 683421 rows and 56798 columns (presolve time = 5s) ...
Presolve removed 705173 rows and 57992 columns
Presolve time: 7.14s
Presolved: 2327 rows, 2008 columns, 54611 nonzeros
Variable types: 0 continuous, 2008 integer (1994 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.1300000e+02   1

## Entrada de datos: Instancia 2x100x2.5x3

In [26]:
datafile="2x100x2.5x3.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

100

In [27]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])
    
ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 707500 rows, 60000 columns and 4543400 nonzeros
Model fingerprint: 0x5042700a
Variable types: 0 continuous, 60000 integer (60000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 6.0000000
Presolve removed 679570 rows and 57010 columns (presolve time = 5s) ...
Presolve removed 703943 rows and 58106 columns
Presolve time: 8.38s
Presolved: 3557 rows, 1894 columns, 76743 nonzeros
Variable types: 0 continuous, 1894 integer (1876 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.8200000e+02   1

## Entrada de datos: Instancia 2x100x2.5x4

In [28]:
datafile="2x100x2.5x4.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

100

In [29]:

p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])
    


ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 707500 rows, 60000 columns and 4543400 nonzeros
Model fingerprint: 0xd35f3d9b
Variable types: 0 continuous, 60000 integer (60000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 7.0000000
Presolve removed 681650 rows and 56850 columns (presolve time = 5s) ...
Presolve removed 705094 rows and 58008 columns
Presolve time: 7.44s
Presolved: 2406 rows, 1992 columns, 55583 nonzeros
Variable types: 0 continuous, 1992 integer (1978 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.3300000e+02   2

   206   221   73.10267   32  272   71.00000   73.17333  3.06%   214   25s
   642   654   72.97986  117  217   71.00000   73.17333  3.06%   177   30s
  2399  2236   73.01159   64  279   71.00000   73.17195  3.06%   104   36s
  2407  2241   72.34345  179  258   71.00000   73.09515  2.95%   104   40s
  2424  2253   72.78220  129  317   71.00000   73.05323  2.89%   103   45s
  2434  2259   72.10173   25  337   71.00000   73.03459  2.87%   103   50s
  2448  2269   72.45904  294  351   71.00000   73.01762  2.84%   102   55s
H 2460  2162                      72.0000000   73.00782  1.40%   102   59s
  2467  2168   72.00000  374  168   72.00000   73.00745  1.40%   118   60s

Cutting planes:
  Gomory: 3
  Cover: 11
  Clique: 14
  MIR: 126
  StrongCG: 49
  Flow cover: 68
  GUB cover: 3
  Zero half: 1
  RLT: 2
  Relax-and-lift: 8

Explored 2472 nodes (338044 simplex iterations) in 60.94 seconds
Thread count was 6 (of 6 available processors)

Solution count 9: 72 71 70 ... -0

Optimal solution fou

## Entrada de datos: Instancia 2x100x2.5x5

In [30]:
datafile="2x100x2.5x5.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

100

In [31]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])
    
ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 707500 rows, 60000 columns and 4543400 nonzeros
Model fingerprint: 0x61e7ec38
Variable types: 0 continuous, 60000 integer (60000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 6.0000000
Presolve removed 687096 rows and 56720 columns (presolve time = 5s) ...
Presolve removed 705204 rows and 58052 columns
Presolve time: 6.34s
Presolved: 2296 rows, 1948 columns, 58027 nonzeros
Found heuristic solution: objective 57.0000000
Variable types: 0 continuous, 1948 integer (1932 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Du

## Entrada de datos: Instancia 2x100x3x1

In [32]:
datafile="2x100x3x1.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

100

In [33]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])
    
ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 707500 rows, 60000 columns and 4543400 nonzeros
Model fingerprint: 0x64e4485b
Variable types: 0 continuous, 60000 integer (60000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 6.0000000
Presolve removed 683386 rows and 56912 columns (presolve time = 5s) ...
Presolve removed 704647 rows and 58046 columns
Presolve time: 7.58s
Presolved: 2853 rows, 1954 columns, 79179 nonzeros
Variable types: 0 continuous, 1954 integer (1926 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.1700000e+02   1

## Entrada de datos: Instancia 2x100x3x2

In [34]:
datafile="2x100x3x2.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

100

In [35]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])
    
ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 707500 rows, 60000 columns and 4543400 nonzeros
Model fingerprint: 0xd2ff9c37
Variable types: 0 continuous, 60000 integer (60000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 7.0000000
Presolve removed 686810 rows and 57088 columns (presolve time = 5s) ...
Presolve removed 705407 rows and 58168 columns
Presolve time: 6.30s
Presolved: 2093 rows, 1832 columns, 45812 nonzeros
Variable types: 0 continuous, 1832 integer (1812 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.5800000e+02   7

## Entrada de datos: Instancia 2x100x3x3

In [36]:
datafile="2x100x3x3.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

100

In [37]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])
    
ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 707500 rows, 60000 columns and 4543400 nonzeros
Model fingerprint: 0xacbf2589
Variable types: 0 continuous, 60000 integer (60000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 7.0000000
Presolve removed 687660 rows and 57092 columns (presolve time = 5s) ...
Presolve removed 704716 rows and 58126 columns
Presolve time: 6.55s
Presolved: 2784 rows, 1874 columns, 80067 nonzeros
Variable types: 0 continuous, 1874 integer (1828 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.9600000e+02   1

## Entrada de datos: Instancia 2x100x3x4

In [38]:
datafile="2x100x3x4.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

100

In [39]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])
    
ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 707500 rows, 60000 columns and 4543400 nonzeros
Model fingerprint: 0x5ba0b8c9
Variable types: 0 continuous, 60000 integer (60000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 8.0000000
Presolve removed 689253 rows and 57416 columns (presolve time = 5s) ...
Presolve removed 705660 rows and 58318 columns
Presolve time: 6.14s
Presolved: 1840 rows, 1682 columns, 36537 nonzeros
Found heuristic solution: objective 49.0000000
Variable types: 0 continuous, 1682 integer (1624 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Du

## Entrada de datos: Instancia 2x100x3x5

In [40]:
datafile="2x100x3x5.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

100

In [41]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])
    
ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 707500 rows, 60000 columns and 4543400 nonzeros
Model fingerprint: 0x5c8adc54
Variable types: 0 continuous, 60000 integer (60000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 10.0000000
Presolve removed 684704 rows and 57086 columns (presolve time = 5s) ...
Presolve removed 705536 rows and 58120 columns
Presolve time: 7.08s
Presolved: 1964 rows, 1880 columns, 45457 nonzeros
Variable types: 0 continuous, 1880 integer (1872 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.9300000e+02   

## Entrada de datos: Instancia 2x100x3.5x1

In [42]:
datafile="2x100x3.5x1.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

100

In [43]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])
    
ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 707500 rows, 60000 columns and 4543400 nonzeros
Model fingerprint: 0xf3edc484
Variable types: 0 continuous, 60000 integer (60000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 8.0000000
Presolve removed 689203 rows and 57360 columns (presolve time = 5s) ...
Presolve removed 704912 rows and 58360 columns
Presolve time: 7.44s
Presolved: 2588 rows, 1640 columns, 79473 nonzeros
Variable types: 0 continuous, 1640 integer (1628 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.3600000e+02   1

## Entrada de datos: Instancia 2x100x3.5x2

In [44]:
datafile="2x100x3.5x2.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

100

In [45]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])
    
ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 707500 rows, 60000 columns and 4543400 nonzeros
Model fingerprint: 0x4106aebd
Variable types: 0 continuous, 60000 integer (60000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 7.0000000
Presolve removed 686970 rows and 57088 columns (presolve time = 5s) ...
Presolve removed 704907 rows and 58162 columns
Presolve time: 6.72s
Presolved: 2593 rows, 1838 columns, 71502 nonzeros
Variable types: 0 continuous, 1838 integer (1812 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.9400000e+02   1

     0     0   61.29581    0  208   60.00000   61.29581  2.16%     -   17s
     0     0   61.29112    0  222   60.00000   61.29112  2.15%     -   17s
     0     0   61.29011    0  221   60.00000   61.29011  2.15%     -   17s
     0     0   61.28990    0  222   60.00000   61.28990  2.15%     -   17s
     0     0   61.28346    0  204   60.00000   61.28346  2.14%     -   17s
     0     0   61.27586    0  218   60.00000   61.27586  2.13%     -   17s
     0     0   61.27506    0  216   60.00000   61.27506  2.13%     -   17s
     0     0   61.27502    0  215   60.00000   61.27502  2.13%     -   17s
     0     0   61.27290    0  218   60.00000   61.27290  2.12%     -   17s
     0     0   61.27271    0  210   60.00000   61.27271  2.12%     -   17s
     0     0   61.27011    0  240   60.00000   61.27011  2.12%     -   17s
     0     0   61.27005    0  253   60.00000   61.27005  2.12%     -   18s
     0     0   61.26760    0  221   60.00000   61.26760  2.11%     -   18s
     0     0   61.26760  

## Entrada de datos: Instancia 2x100x3.5x3

In [46]:
datafile="2x100x3.5x3.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

100

In [47]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])
    
ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 707500 rows, 60000 columns and 4543400 nonzeros
Model fingerprint: 0x8e0a56e0
Variable types: 0 continuous, 60000 integer (60000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 9.0000000
Presolve removed 685267 rows and 57214 columns (presolve time = 5s) ...
Presolve removed 705482 rows and 58242 columns
Presolve time: 7.77s
Presolved: 2018 rows, 1758 columns, 47864 nonzeros
Variable types: 0 continuous, 1758 integer (1730 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.7100000e+02   1

## Entrada de datos: Instancia 2x100x3.5x4

In [48]:
datafile="2x100x3.5x4.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

100

In [49]:

p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])
    


ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 707500 rows, 60000 columns and 4543400 nonzeros
Model fingerprint: 0xa3a48205
Variable types: 0 continuous, 60000 integer (60000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 7.0000000
Presolve removed 686770 rows and 57358 columns (presolve time = 5s) ...
Presolve removed 705304 rows and 58270 columns
Presolve time: 7.32s
Presolved: 2196 rows, 1730 columns, 47854 nonzeros
Variable types: 0 continuous, 1730 integer (1702 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.9700000e+02   9

     0     0   61.46918    0  187   60.00000   61.46918  2.45%     -   13s
     0     0   61.46918    0  192   60.00000   61.46918  2.45%     -   13s
     0     0   61.46918    0  202   60.00000   61.46918  2.45%     -   13s
     0     0   61.46916    0  215   60.00000   61.46916  2.45%     -   13s
     0     0   61.46864    0  204   60.00000   61.46864  2.45%     -   13s
     0     0   61.46861    0  214   60.00000   61.46861  2.45%     -   13s
     0     0   61.45380    0  188   60.00000   61.45380  2.42%     -   13s
     0     0   61.45042    0  172   60.00000   61.45042  2.42%     -   14s
     0     0   61.44809    0  188   60.00000   61.44809  2.41%     -   14s
     0     0   61.44806    0  180   60.00000   61.44806  2.41%     -   14s
     0     0   61.44579    0  176   60.00000   61.44579  2.41%     -   14s
     0     0   61.44413    0  188   60.00000   61.44413  2.41%     -   14s
     0     0   61.44348    0  200   60.00000   61.44348  2.41%     -   14s
     0     0   61.44074  

## Entrada de datos: Instancia 2x100x3.5x5

In [50]:
datafile="2x100x3.5x5.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

100

In [51]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])
    
ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 707500 rows, 60000 columns and 4543400 nonzeros
Model fingerprint: 0x36a335be
Variable types: 0 continuous, 60000 integer (60000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 6.0000000
Presolve removed 686481 rows and 57072 columns (presolve time = 5s) ...
Presolve removed 704948 rows and 58132 columns
Presolve time: 7.08s
Presolved: 2552 rows, 1868 columns, 63928 nonzeros
Variable types: 0 continuous, 1868 integer (1828 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.9100000e+02   1

## Entrada de datos: Instancia 2x100x4x1

In [52]:
datafile="2x100x4x1.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

100

In [53]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])
    
ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 707500 rows, 60000 columns and 4543400 nonzeros
Model fingerprint: 0xf99f3af3
Variable types: 0 continuous, 60000 integer (60000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 8.0000000
Presolve removed 690968 rows and 57762 columns (presolve time = 5s) ...
Presolve removed 706037 rows and 58600 columns
Presolve time: 5.90s
Presolved: 1463 rows, 1400 columns, 29155 nonzeros
Found heuristic solution: objective 44.0000000
Variable types: 0 continuous, 1400 integer (1372 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Du

## Entrada de datos: Instancia 2x100x4x2

In [54]:
datafile="2x100x4x2.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

100

In [55]:

p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])
    
ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 707500 rows, 60000 columns and 4543400 nonzeros
Model fingerprint: 0xec7a6ed7
Variable types: 0 continuous, 60000 integer (60000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 4.0000000
Presolve removed 690778 rows and 57702 columns (presolve time = 5s) ...
Presolve removed 705591 rows and 58550 columns
Presolve time: 6.56s
Presolved: 1909 rows, 1450 columns, 50749 nonzeros
Found heuristic solution: objective 49.0000000
Variable types: 0 continuous, 1450 integer (1430 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Du

     0     0   57.66282    0  137   56.00000   57.66282  2.97%     -   11s
     0     2   57.66279    0  134   56.00000   57.66279  2.97%     -   11s
  1334  1003   57.29014   29   81   56.00000   57.64641  2.94%  76.5   15s
  1384  1043   57.00000   26   64   56.00000   57.00000  1.79%  95.5   20s
  2121  1404   57.00000   37   58   56.00000   57.00000  1.79%   101   26s
  3655  1965   57.00000   80  121   56.00000   57.00000  1.79%   109   30s
  5552  2341   57.00000   83  106   56.00000   57.00000  1.79%   135   37s
  6238  2543     cutoff  120        56.00000   57.00000  1.79%   139   40s
  7916  2975   57.00000   83  128   56.00000   57.00000  1.79%   152   45s
  9746  3473     cutoff   74        56.00000   57.00000  1.79%   157   52s
 10394  3744     cutoff  110        56.00000   57.00000  1.79%   162   56s
 12406  4083     cutoff   94        56.00000   57.00000  1.79%   169   63s
 12991  4111   57.00000  104   67   56.00000   57.00000  1.79%   173   66s
 13233  4305     cutoff  

 150367 15177   57.00000   75   80   56.00000   57.00000  1.79%   207  600s
 151650 15129   57.00000   82   83   56.00000   57.00000  1.79%   207  607s
 152663 15197     cutoff   92        56.00000   57.00000  1.79%   208  610s
 154585 15329     cutoff  106        56.00000   57.00000  1.79%   208  620s
 155131 15580   57.00000   80   92   56.00000   57.00000  1.79%   208  627s
 156243 15634     cutoff   72        56.00000   57.00000  1.79%   208  630s
 158175 15831   57.00000   69   95   56.00000   57.00000  1.79%   209  637s
 159115 15908     cutoff   87        56.00000   57.00000  1.79%   209  641s
 160022 15986   57.00000   75  128   56.00000   57.00000  1.79%   209  645s
 161916 16001     cutoff   82        56.00000   57.00000  1.79%   209  652s
 163069 16042     cutoff   78        56.00000   57.00000  1.79%   209  658s
 163306 16081     cutoff   72        56.00000   57.00000  1.79%   209  661s
 165438 16103     cutoff   73        56.00000   57.00000  1.79%   209  668s
 166543 1612

 279066 19290     cutoff   82        56.00000   57.00000  1.79%   221 1191s
 279946 19435   57.00000   78  129   56.00000   57.00000  1.79%   221 1196s
 281256 19447     cutoff   77        56.00000   57.00000  1.79%   221 1206s
 282663 19735     cutoff  100        56.00000   57.00000  1.79%   221 1213s
 283693 19767   57.00000   57   89   56.00000   57.00000  1.79%   221 1216s
 284015 19912   57.00000   58   85   56.00000   57.00000  1.79%   221 1220s
 284826 20066   57.00000   93   74   56.00000   57.00000  1.79%   221 1225s
 287451 20156   57.00000   97   43   56.00000   57.00000  1.79%   221 1237s
 288126 20241   57.00000   70  132   56.00000   57.00000  1.79%   221 1240s
 290346 20324     cutoff   69        56.00000   57.00000  1.79%   221 1246s
 291775 20552     cutoff   86        56.00000   57.00000  1.79%   221 1253s
 292745 20565   57.00000   80   10   56.00000   57.00000  1.79%   221 1260s
 292844 20645     cutoff   81        56.00000   57.00000  1.79%   221 1265s
 294766 2081

 402623 25344   57.00000   87   73   56.00000   57.00000  1.79%   226 1790s
 403374 25380     cutoff   97        56.00000   57.00000  1.79%   226 1795s
 406175 25358   57.00000   70  103   56.00000   57.00000  1.79%   226 1802s
 407214 25369     cutoff   80        56.00000   57.00000  1.79%   226 1812s
 407397 25353   57.00000   86  101   56.00000   57.00000  1.79%   226 1815s
 409812 25525   57.00000   83   61   56.00000   57.00000  1.79%   226 1822s
 410555 25591     cutoff   88        56.00000   57.00000  1.79%   226 1826s
 412245 25763     cutoff   88        56.00000   57.00000  1.79%   226 1833s
 413384 25883     cutoff   73        56.00000   57.00000  1.79%   226 1836s
 414568 25933     cutoff   96        56.00000   57.00000  1.79%   226 1840s
 415376 25951     cutoff   98        56.00000   57.00000  1.79%   226 1845s
 415952 26102     cutoff   91        56.00000   57.00000  1.79%   226 1850s
 418780 26135     cutoff   80        56.00000   57.00000  1.79%   226 1857s
 419845 2615

 525412 29338     cutoff   84        56.00000   57.00000  1.79%   229 2391s
 526612 29435   57.00000   96   80   56.00000   57.00000  1.79%   229 2395s
 527828 29558     cutoff   92        56.00000   57.00000  1.79%   229 2401s
 528658 29652   57.00000   81   50   56.00000   57.00000  1.79%   229 2405s
 530496 29722     cutoff  101        56.00000   57.00000  1.79%   229 2413s
 531193 29876   57.00000   80  110   56.00000   57.00000  1.79%   229 2417s
 532371 30019     cutoff   80        56.00000   57.00000  1.79%   229 2421s
 533120 30089     cutoff   91        56.00000   57.00000  1.79%   229 2427s
 533674 30225   57.00000   76   42   56.00000   57.00000  1.79%   229 2431s
 534782 30269   57.00000   82   69   56.00000   57.00000  1.79%   229 2435s
 537716 30119   57.00000   79   41   56.00000   57.00000  1.79%   229 2441s
 538529 30072     cutoff   78        56.00000   57.00000  1.79%   229 2445s
 540115 30115     cutoff   88        56.00000   57.00000  1.79%   229 2451s
 541191 3010

 655682 30986   57.00000   69   94   56.00000   57.00000  1.79%   231 2989s
 656028 30970     cutoff   70        56.00000   57.00000  1.79%   231 2992s
 657282 30906   57.00000  107   75   56.00000   57.00000  1.79%   231 2996s
 658418 30932     cutoff   98        56.00000   57.00000  1.79%   231 3000s
 659606 30913     cutoff   90        56.00000   57.00000  1.79%   231 3008s
 659743 30969   57.00000   83   93   56.00000   57.00000  1.79%   231 3011s
 660823 30973   57.00000  107   79   56.00000   57.00000  1.79%   231 3015s
 661098 31022   57.00000   71   73   56.00000   57.00000  1.79%   231 3027s
 664316 31112   57.00000   93   95   56.00000   57.00000  1.79%   231 3033s
 665318 31208     cutoff   83        56.00000   57.00000  1.79%   231 3038s
 666386 31259   57.00000   59   98   56.00000   57.00000  1.79%   231 3043s
 666931 31302     cutoff   65        56.00000   57.00000  1.79%   231 3047s
 668128 31373   57.00000   79   71   56.00000   57.00000  1.79%   231 3051s
 669299 3134

  Zero half: 102
  Relax-and-lift: 16

Explored 767849 nodes (179847269 simplex iterations) in 3600.38 seconds
Thread count was 6 (of 6 available processors)

Solution count 7: 56 55 54 ... -0

Time limit reached
Best objective 5.600000000000e+01, best bound 5.700000000000e+01, gap 1.7857%


## Entrada de datos: Instancia 2x100x4x3

In [56]:
datafile="2x100x4x3.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

100

In [57]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])
    
ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 707500 rows, 60000 columns and 4543400 nonzeros
Model fingerprint: 0x5b0aca6e
Variable types: 0 continuous, 60000 integer (60000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 6.0000000
Presolve removed 704367 rows and 58484 columns (presolve time = 5s) ...
Presolve removed 706057 rows and 58664 columns
Presolve time: 5.40s
Presolved: 1443 rows, 1336 columns, 30230 nonzeros
Variable types: 0 continuous, 1336 integer (1328 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.2600000e+02   5

## Entrada de datos: Instancia 2x100x4x4

In [58]:
datafile="2x100x4x4.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

100

In [59]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])
    
ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 707500 rows, 60000 columns and 4543400 nonzeros
Model fingerprint: 0x6a829eeb
Variable types: 0 continuous, 60000 integer (60000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 8.0000000
Presolve removed 691089 rows and 57828 columns (presolve time = 5s) ...
Presolve removed 705960 rows and 58606 columns
Presolve time: 5.91s
Presolved: 1540 rows, 1394 columns, 32665 nonzeros
Variable types: 0 continuous, 1394 integer (1368 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.3100000e+02   6

## Entrada de datos: Instancia 2x100x4x5

In [60]:
datafile="2x100x4x5.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

100

In [61]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])
    
ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 707500 rows, 60000 columns and 4543400 nonzeros
Model fingerprint: 0x5793a51a
Variable types: 0 continuous, 60000 integer (60000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 7.0000000
Presolve removed 690187 rows and 57790 columns (presolve time = 5s) ...
Presolve removed 705832 rows and 58576 columns
Presolve time: 5.99s
Presolved: 1668 rows, 1424 columns, 34730 nonzeros
Variable types: 0 continuous, 1424 integer (1402 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.4900000e+02   6

## Entrada de datos: Instancia 2x100x4.5x1

In [62]:
datafile="2x100x4.5x1.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

100

In [63]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 707500 rows, 60000 columns and 4543400 nonzeros
Model fingerprint: 0x96404d72
Variable types: 0 continuous, 60000 integer (60000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 6.0000000
Presolve removed 705651 rows and 58682 columns (presolve time = 5s) ...
Presolve removed 705822 rows and 58728 columns
Presolve time: 5.33s
Presolved: 1678 rows, 1272 columns, 36273 nonzeros
Variable types: 0 continuous, 1272 integer (1260 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.3200000e+02   7

## Entrada de datos: Instancia 2x100x4.5x2

In [64]:
datafile="2x100x4.5x2.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

100

In [65]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])
    
ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 707500 rows, 60000 columns and 4543400 nonzeros
Model fingerprint: 0xfdab513b
Variable types: 0 continuous, 60000 integer (60000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 9.0000000
Presolve removed 691386 rows and 57982 columns (presolve time = 5s) ...
Presolve removed 705690 rows and 58744 columns
Presolve time: 6.58s
Presolved: 1810 rows, 1256 columns, 44965 nonzeros
Found heuristic solution: objective 42.0000000
Variable types: 0 continuous, 1256 integer (1242 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Du

## Entrada de datos: Instancia 2x100x4.5x3

In [66]:
datafile="2x100x4.5x3.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

100

In [67]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])
    
ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 707500 rows, 60000 columns and 4543400 nonzeros
Model fingerprint: 0x702ba1d8
Variable types: 0 continuous, 60000 integer (60000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 9.0000000
Presolve removed 692244 rows and 58020 columns (presolve time = 5s) ...
Presolve removed 706032 rows and 58762 columns
Presolve time: 5.81s
Presolved: 1468 rows, 1238 columns, 29401 nonzeros
Found heuristic solution: objective 42.0000000
Variable types: 0 continuous, 1238 integer (1222 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Du

## Entrada de datos: Instancia 2x100x4.5x4

In [68]:
datafile="2x100x4.5x4.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

100

In [69]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])
    
ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 707500 rows, 60000 columns and 4543400 nonzeros
Model fingerprint: 0xba481ab7
Variable types: 0 continuous, 60000 integer (60000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 7.0000000
Presolve removed 703833 rows and 58532 columns (presolve time = 5s) ...
Presolve removed 705787 rows and 58806 columns
Presolve time: 5.60s
Presolved: 1713 rows, 1194 columns, 45997 nonzeros
Found heuristic solution: objective 43.0000000
Variable types: 0 continuous, 1194 integer (1184 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Du

## Entrada de datos: Instancia 2x100x4.5x5

In [70]:
datafile="2x100x4.5x5.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

100

In [71]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])
    
ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 707500 rows, 60000 columns and 4543400 nonzeros
Model fingerprint: 0xd563cdb8
Variable types: 0 continuous, 60000 integer (60000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 6.0000000
Presolve removed 692363 rows and 58024 columns (presolve time = 5s) ...
Presolve removed 706108 rows and 58750 columns
Presolve time: 5.52s
Presolved: 1392 rows, 1250 columns, 26925 nonzeros
Variable types: 0 continuous, 1250 integer (1236 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.8300000e+02   9

## Entrada de datos: Instancia 2x100x5x1

In [72]:
datafile="2x100x5x1.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

100

In [73]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])
    
ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 707500 rows, 60000 columns and 4543400 nonzeros
Model fingerprint: 0x5ed6e258
Variable types: 0 continuous, 60000 integer (60000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 9.0000000
Presolve removed 706382 rows and 58878 columns
Presolve time: 4.89s
Presolved: 1118 rows, 1122 columns, 24237 nonzeros
Variable types: 0 continuous, 1122 integer (1116 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.1800000e+02   8.567252e+02   0.000000e+00      5s
     294    4.6448809e+01   0.000000e

## Entrada de datos: Instancia 2x100x5x2

In [74]:
datafile="2x100x5x2.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

100

In [75]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])
    
ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 707500 rows, 60000 columns and 4543400 nonzeros
Model fingerprint: 0xe7f522d8
Variable types: 0 continuous, 60000 integer (60000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 6.0000000
Presolve removed 705185 rows and 58712 columns (presolve time = 5s) ...
Presolve removed 706306 rows and 58808 columns
Presolve time: 5.34s
Presolved: 1194 rows, 1192 columns, 28180 nonzeros
Variable types: 0 continuous, 1192 integer (1178 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0500000e+02   6

## Entrada de datos: Instancia 2x100x5x3

In [76]:
datafile="2x100x5x3.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

100

In [77]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])
    
ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 707500 rows, 60000 columns and 4543400 nonzeros
Model fingerprint: 0x5839f8ac
Variable types: 0 continuous, 60000 integer (60000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 9.0000000
Presolve removed 706367 rows and 58954 columns
Presolve time: 4.87s
Presolved: 1133 rows, 1046 columns, 20618 nonzeros
Variable types: 0 continuous, 1046 integer (1028 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.1900000e+02   3.228455e+02   0.000000e+00      5s
     479    4.5399355e+01   0.000000e

## Entrada de datos: Instancia 2x100x5x4

In [78]:
datafile="2x100x5x4.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

100

In [79]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 707500 rows, 60000 columns and 4543400 nonzeros
Model fingerprint: 0x1b55a956
Variable types: 0 continuous, 60000 integer (60000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 5.0000000
Presolve removed 705960 rows and 58736 columns (presolve time = 5s) ...
Presolve removed 706134 rows and 58792 columns
Presolve time: 5.45s
Presolved: 1366 rows, 1208 columns, 27336 nonzeros
Variable types: 0 continuous, 1208 integer (1192 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.9000000e+01   4

## Entrada de datos: Instancia 2x100x5x5

In [80]:
datafile="2x100x5x5.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

100

In [81]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])
    
ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 707500 rows, 60000 columns and 4543400 nonzeros
Model fingerprint: 0x85613d95
Variable types: 0 continuous, 60000 integer (60000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 5.0000000
Presolve removed 705967 rows and 58750 columns (presolve time = 5s) ...
Presolve removed 706081 rows and 58756 columns
Presolve time: 5.27s
Presolved: 1419 rows, 1244 columns, 29624 nonzeros
Variable types: 0 continuous, 1244 integer (1232 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.5100000e+02   9